In [1]:
# code for loading the format for the notebook
import os

# path : store the current path to convert back to it later
path = os.getcwd()
os.chdir('../notebook_format')
from formats import load_style
load_style( css_style = 'custom2.css' )

In [2]:
os.chdir(path)
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 8, 6 # change default figure size

# 1. magic to print version
# 2. magic so that the notebook will reload external python modules
%load_ext watermark
%load_ext autoreload 
%autoreload 2

# note that Convolution2D is the same as Conv2D
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D

%watermark -a 'Ethen' -d -t -v -p numpy,pandas,matplotlib,keras

Using TensorFlow backend.


Ethen 2016-12-12 17:35:18 

CPython 3.5.2
IPython 4.2.0

numpy 1.11.2
pandas 0.18.1
matplotlib 1.5.1
keras 1.1.1


# Convolutional Network

In [3]:
# loading the mnist dataset as an example
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print( 'X_train shape:', X_train.shape )
print( X_train.shape[0], 'train samples' )
print( X_test.shape[0] , 'test samples' )

X_train shape: (60000, 28, 28)
60000 train samples
10000 test samples


In [4]:
# one-hot encode the class (target) vectors
n_class = 10
y_train = np_utils.to_categorical(y_train, n_class)
y_test  = np_utils.to_categorical(y_test , n_class)
print('y_train shape:', y_train.shape)

# input image dimensions (n_input), note that we add another 1 to the dimension
# to represent the depth (RGB color), and it is at the very beginning (index 1)
# ,this is just the default way of keras representing the input dimension
n_input = 1, 28, 28
X_train = X_train.reshape(X_train.shape[0], *n_input)
X_test  = X_test.reshape(X_test.shape[0], *n_input)
X_train = X_train.astype('float32')
X_test  = X_test.astype('float32')

# images takes values between 0 - 255, we can normalize it
# by dividing very number by 255
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)

y_train shape: (60000, 10)
X_train shape: (60000, 1, 28, 28)


Example of api call of a Convnet:

```python
# apply a 64 3x3 filters on a 256x256x3 image:
model = Sequential()

# the first three argument are for the filters,
# then we specify the `border_mode` to be 'same' so we get
# the same width and height for the input (it will automatically do zero-padding)
# and since this is the first layer we need to specify the input shape of the image
model.add( Convolution2D( 64, 3, 3, border_mode = 'same', 
                          input_shape = (3, 256, 256) ) )

# the output shape == (None, 64, 256, 256), where None just stands
# for the number of data points
# the stride is called `subsample`, and the default is 1

# some activation function after conv layer
model.add(Activation('relu'))

# pooling layer, we specify the size of the filters for the pooling layer
# e.g. n_pool = 2, 2, if `stride` is None, it will default to pool_size
model.add(MaxPooling2D(pool_size = n_pool))

# before calling the fully-connected layers, we'll have to flatten it
# e.g. model.add(Convolution2D(64, 3, 3, border_mode='same', input_shape=(3, 32, 32)))
# now: model's output shape == (None, 64, 32, 32)
# model.add(Flatten())
# now: model.output_shape == (None, 65536)
model.add(Flatten())
model.add(Dense(n_class))
model.add(Activation('softmax'))

```

Let's now try constructing the whole thing.

In [5]:
# network parameter
# 1. number of convolutional filters to use
# 2. each filters' size
# 3. pooling filters' size
n_conv_size = 3
n_conv_num = 64
n_conv = n_conv_num, n_conv_size, n_conv_size
n_pool = 2, 2

# model parameter
n_class = 10
n_input = 1, 28, 28
n_epoch = 12
n_batch_size = 2056

<p>
<div class="alert alert-danger">
The following code chunk takes A WHILE on a laptop!!
</div>

In [6]:
model = Sequential()
model.add(Convolution2D(*n_conv, border_mode = 'same', input_shape = n_input))
model.add(Activation('relu'))
model.add(Convolution2D(*n_conv, border_mode = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = n_pool))
model.add(Flatten())
model.add(Dense(n_class))
model.add(Activation('softmax'))

model.compile( loss = 'categorical_crossentropy',
               optimizer = 'adam',
               metrics = ['accuracy'] )

# use verbose one to print progress bars
model.fit( X_train, y_train, 
           batch_size = n_batch_size, 
           nb_epoch = n_epoch,
           verbose = 1, 
           validation_data = (X_test, y_test) )

# evaluating the score, categorical cross entropy error and accuracy
score = model.evaluate(X_test, y_test, verbose = 0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 503s - loss: 0.9013 - acc: 0.7702 - val_loss: 0.3490 - val_acc: 0.9043
Epoch 2/12
60000/60000 [==============================] - 504s - loss: 0.2818 - acc: 0.9191 - val_loss: 0.2039 - val_acc: 0.9436
Epoch 3/12
60000/60000 [==============================] - 503s - loss: 0.1722 - acc: 0.9511 - val_loss: 0.1216 - val_acc: 0.9648
Epoch 4/12
60000/60000 [==============================] - 500s - loss: 0.1064 - acc: 0.9697 - val_loss: 0.0793 - val_acc: 0.9758
Epoch 5/12
60000/60000 [==============================] - 504s - loss: 0.0775 - acc: 0.9779 - val_loss: 0.0663 - val_acc: 0.9809
Epoch 6/12
60000/60000 [==============================] - 618s - loss: 0.0638 - acc: 0.9810 - val_loss: 0.0554 - val_acc: 0.9825
Epoch 7/12
60000/60000 [==============================] - 651s - loss: 0.0546 - acc: 0.9841 - val_loss: 0.0510 - val_acc: 0.9838
Epoch 8/12
60000/60000 [=======================

## Reference

- [Keras Example: mnist_cnn example](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py)